In [ ]:
# dependencies

import pandas as pd
import numpy as np
import os


import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy import Column, Float, Integer, String, Date
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [ ]:
engine = create_engine("sqlite:///EatingOutNYC.sqlite")
conn = engine.connect()

In [ ]:
# create a class for the Restaurants table

class Restaurants(Base):
    __tablename__ = "Restaurants"
    
    Restaurant_ID = Column(Integer, primary_key = True)
    CAMIS = Column(Integer)
    DBA = Column(String)
    BORO = Column(String)
    BUILDING = Column(Integer)
    STREET = Column(String)
    ZIPCODE = Column(Integer)
    PHONE = Column(Integer)
    CUISINE_DESCRIPTION = Column(String)
    CUISINE = Column(String)
    LIC_STATUS = Column(String)
    SWC_TYPE = Column(String)
    LATITUDE = Column(String)
    LONGITUDE = Column(String)
    ISSUANCE = Column(String)
    ISSUANCE_DD = Column(String)
    
    
    def __repr__(self):
        return f"id = {self.id}, CAMIS = {self.CAMIS}, DBA = {self.DBA}, BORO = {self.BORO}, BUILDING = {self.BUILDING}, STREET = {self.STREET}, ZIPCODE = {self.ZIPCODE}, PHONE = {self.PHONE}, CUISINE_DESCRIPTION = {self.CUISINE_DESCRIPTION}, CUISINE = {self.CUISINE}, LIC_STATUS = {self.LIC_STATUS}, SWC_TYPE = {self.SWC_TYPE}, LATITUDE = {self.LATITUDE}, LONGITUDE = {self.LONGITUDE}, ISSUANCE = {self.ISSUANCE}, ISSUANCE_DD = {self.ISSUANCE_DD}"   



In [ ]:
# create a class for the Inspections table

class Inspections(Base):
    __tablename__ = "Inspections"
    
    Inspections_ID = Column(Integer, primary_key = True)
    CAMIS = Column(Integer)
    INSPECTION_DATE = Column(String)
    INSPECTION_DAY = Column(Integer)
    INSPECTION_MONTH = Column(Integer)
    INSPECTION_WEEKDAY = Column(Integer)
    INSPECTION_YEAR = Column(Integer)
    ACTION = Column(String)
    VIOLATION_CODE = Column(String)
    VIOLATION_DESCRIPTION = Column(String)
    CRITICAL_FLAG = Column(String)
    SCORE = Column(Integer)
    GRADE = Column(String)
    INSPECTION_TYPE = Column(String) 
    
    def __repr__(self):
        return f"id = {self.id}, CAMIS = {self.CAMIS}, INSPECTION_DATE = {self.INSPECTION_DATE}, INSPECTION_DAY = {self.INSPECTION_DAY}, INSPECTION_MONTH = {self.INSPECTION_MONTH}, INSPECTION_WEEKDAY = {self.INSPECTION_WEEKDAY}, INSPECTION_YEAR = {self.INSPECTION_YEAR}, ACTION = {self.ACTION}, VIOLATION_CODE = {self.VIOLATION_CODE}, VIOLATION_DESCRIPTION = {self.VIOLATION_DESCRIPTION}, CRITICAL_FLAG = {self.CRITICAL_FLAG}, SCORE = {self.SCORE}, GRADE = {self.GRADE}, INSPECTION_TYPE = {self.INSPECTION_TYPE}"   



In [ ]:
Base.metadata.create_all(engine)

In [ ]:
# load the Restaurants csv file into a dict
Restaurants_df = pd.read_csv('Restaurants.csv')
Restaurants_data = Restaurants_df.to_dict(orient='records')
#print(Restaurants_data[:5])

In [ ]:
# load the Inspections csv file into a dict
Inspections_df = pd.read_csv('Inspections.csv')
Inspections_data = Inspections_df.to_dict(orient='records')
#print(Inspections_data[:5])

In [ ]:
metadata = MetaData(bind = engine)
metadata.reflect()

In [ ]:
Restaurants = sqlalchemy.Table('Restaurants', metadata, autoload=True)

In [ ]:
Inspections = sqlalchemy.Table('Inspections', metadata, autoload=True)

In [ ]:
# empty out any data that may exist in Restaurants table from prior run 
conn.execute(Restaurants.delete())

In [ ]:
# empty out any data that may exist in Inspections table from prior run 
conn.execute(Inspections.delete())

In [ ]:
# populate the Restaurants table
conn.execute(Restaurants.insert(), Restaurants_data)

In [ ]:
# populate the Inspections table
conn.execute(Inspections.insert(), Inspections_data)

In [ ]:
# show the first 5 rows from the Restaurants table. 
conn.execute("select * from Restaurants limit 5").fetchall()

In [ ]:
# count how many Restaurants are in the Restaurants table
Restaurantscount = conn.execute("select count(*) from Restaurants;")
print(Restaurantscount.scalar())

In [ ]:
# show the first 5 rows from the Inspections table. 
conn.execute("select * from Inspections limit 5").fetchall()

In [ ]:
# count how many Inspections are in the Inspections table
Inspectionscount = conn.execute("select count(*) from Inspections;")
print(Inspectionscount.scalar())

In [ ]:
# let's test an example of composite SQL query

conn.execute("SELECT Inspections.VIOLATION_DESCRIPTION FROM Restaurants INNER JOIN Inspections ON Restaurants.CAMIS = Inspections.CAMIS WHERE (((Restaurants.CAMIS)=40662196));").fetchall()

In [ ]:
###################
# VIOLATIONS DATA #
###################

In [ ]:
# Add columns for the violation types

conn.execute("ALTER TABLE Inspections ADD COLUMN MICE INT")
conn.execute("ALTER TABLE Inspections ADD COLUMN RATS INT")
conn.execute("ALTER TABLE Inspections ADD COLUMN ROACHES INT")
conn.execute("ALTER TABLE Inspections ADD COLUMN FLIES INT")
conn.execute("ALTER TABLE Inspections ADD COLUMN HANDS INT")
conn.execute("ALTER TABLE Inspections ADD COLUMN SANITIZE INT")


In [ ]:
# Populate the new columns where they appear in the violation description
conn.execute("UPDATE Inspections SET MICE=1 WHERE Inspections.VIOLATION_DESCRIPTION LIKE '%mice%'")
conn.execute("UPDATE Inspections SET RATS=1 WHERE Inspections.VIOLATION_DESCRIPTION LIKE '%rats%'")
conn.execute("UPDATE Inspections SET ROACHES=1 WHERE Inspections.VIOLATION_DESCRIPTION LIKE '%roach%'")
conn.execute("UPDATE Inspections SET FLIES=1 WHERE Inspections.VIOLATION_DESCRIPTION LIKE '%flies%'")
conn.execute("UPDATE Inspections SET HANDS=1 WHERE Inspections.VIOLATION_DESCRIPTION LIKE '%hand%'")
conn.execute("UPDATE Inspections SET SANITIZE=1 WHERE Inspections.VIOLATION_DESCRIPTION LIKE '%sanitiz%'")

#conn.execute("SELECT * FROM Inspections limit 5").fetchall()


In [ ]:
# Query for the data to check the right columns are returned

#conn.execute("SELECT Restaurants.CAMIS, Restaurants.DBA, Restaurants.BORO, Restaurants.CUISINE, Restaurants.LIC_STATUS, Inspections.INSPECTION_DATE, Inspections.SCORE, Inspections.GRADE, Inspections.CRITICAL_FLAG, Inspections.MICE, Inspections.RATS, Inspections.ROACHES, Inspections.FLIES, Inspections.HANDS, Inspections.SANITIZE FROM Restaurants INNER JOIN Inspections ON Restaurants.CAMIS=Inspections.CAMIS LIMIT 5").fetchall()


In [ ]:
# Make a DataFrame and Review

violations = conn.execute("SELECT Restaurants.CAMIS, Restaurants.DBA, Restaurants.BORO, Restaurants.CUISINE, Restaurants.LIC_STATUS, Inspections.INSPECTION_DATE, Inspections.SCORE, Inspections.GRADE, Inspections.CRITICAL_FLAG, Inspections.MICE, Inspections.RATS, Inspections.ROACHES, Inspections.FLIES, Inspections.HANDS, Inspections.SANITIZE FROM Restaurants INNER JOIN Inspections ON Restaurants.CAMIS=Inspections.CAMIS").fetchall()
violations_df = pd.DataFrame(violations, columns=["CAMIS", "DBA","BORO","CUISINE","LIC_STATUS","INSPECTION_DATE","SCORE","GRADE","CRITICAL_FLAG","MICE","RATS","ROACHES","FLIES","HANDS","SANITIZE"])

# Convert NaNs to 0 and confirm numeric columns are numeric variable type
violations_df = violations_df.fillna(0)
violations_df[violations_df.columns] = violations_df[violations_df.columns].apply(pd.to_numeric, errors="ignore")

# violations_df.head()


In [ ]:
# Orient the data to what we need to graph in Plotly

# SELECT VIOLATIONS BY BORO

boro_df = violations_df[["BORO","MICE","RATS","ROACHES","FLIES","HANDS","SANITIZE"]]
boro_df = boro_df[boro_df.BORO != 'Missing']
# boro_df.shape
# boro_df.head()

boro_df = boro_df.groupby(["BORO"], as_index=False).sum()
boro_df["TOTAL"] = boro_df["MICE"] + boro_df["RATS"] + boro_df["ROACHES"] + boro_df["FLIES"] + boro_df["HANDS"] + boro_df["SANITIZE"]

boro_df["MICEpct"] = (boro_df["MICE"] / boro_df["TOTAL"])*100
boro_df["RATSpct"] = (boro_df["RATS"] / boro_df["TOTAL"])*100
boro_df["ROACHESpct"] = (boro_df["ROACHES"] / boro_df["TOTAL"])*100
boro_df["FLIESpct"] = (boro_df["FLIES"] / boro_df["TOTAL"])*100
boro_df["HANDSpct"] = (boro_df["HANDS"] / boro_df["TOTAL"])*100
boro_df["SANITIZEpct"] = (boro_df["SANITIZE"] / boro_df["TOTAL"])*100
boro_df["TOTALpct"] = boro_df["MICEpct"] + boro_df["RATSpct"] + boro_df["ROACHESpct"] + boro_df["FLIESpct"] + boro_df["HANDSpct"] + boro_df["SANITIZEpct"]

# boro_df

# EXPORT BORO VIOLATIONS DATA TO JSON FILE
boro_df.to_json("VIOL_BORO.json", orient="records")


In [ ]:
# SELECT VIOLATIONS BY CUISINE TYPE

cuisine_df = violations_df[["CUISINE","MICE","RATS","ROACHES","FLIES","HANDS","SANITIZE"]]
#cuisine_df.head()

cuisine_df = cuisine_df.groupby("CUISINE", as_index=False).sum()
cuisine_df["TOTAL"] = cuisine_df["MICE"] + cuisine_df["RATS"] + cuisine_df["ROACHES"] + cuisine_df["FLIES"] + cuisine_df["HANDS"] + cuisine_df["SANITIZE"]

cuisine_df["MICEpct"] = (cuisine_df["MICE"] / cuisine_df["TOTAL"])*100
cuisine_df["RATSpct"] = (cuisine_df["RATS"] / cuisine_df["TOTAL"])*100
cuisine_df["ROACHESpct"] = (cuisine_df["ROACHES"] / cuisine_df["TOTAL"])*100
cuisine_df["FLIESpct"] = (cuisine_df["FLIES"] / cuisine_df["TOTAL"])*100
cuisine_df["HANDSpct"] = (cuisine_df["HANDS"] / cuisine_df["TOTAL"])*100
cuisine_df["SANITIZEpct"] = (cuisine_df["SANITIZE"] / cuisine_df["TOTAL"])*100

cuisine_df["TOTALpct"] = cuisine_df["MICEpct"] + cuisine_df["RATSpct"] + cuisine_df["ROACHESpct"] + cuisine_df["FLIESpct"] + cuisine_df["HANDSpct"] + cuisine_df["SANITIZEpct"]
cuisine_df

#cuisine_df.to_json("VIOL_CUISINE.json", orient="records")
